In [ ]:
# --- Imports ---
import tensorflow as tf
from tensorflow.keras.layers import Layer, Dense, Input, Dropout # Conv2D etc. removed as not needed for NLP
from tensorflow.keras.models import Model # Sequential might not be needed
from tensorflow.keras.optimizers import AdamW # Standard optimizer for BERT
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.utils import to_categorical
from transformers import BertTokenizer, TFBertModel, BertConfig # Key imports for BERT
import datasets as hf_datasets # Hugging Face datasets library for easy loading
import numpy as np
import pandas as pd
import random
import time
import os
import gc

# --- Configuration (general and NLP-specific) ---
RAND_SEED = 42
NUM_RUNS = 1 # Start with 1, BERT training takes longer
EPOCHS_NLP = 3 # Fine-tuning BERT usually requires fewer epochs
PATIENCE_NLP = 2 # For EarlyStopping
BATCH_SIZE_NLP = 16 # May need to be reduced (e.g., 8 or 4) depending on GPU/TPU memory
MAX_LENGTH = 128  # Maximum sequence length for tokenizer
BERT_MODEL_NAME = 'bert-base-uncased' # Popular BERT model choice
LEARNING_RATE_BERT = 2e-5 # Typical learning rate for fine-tuning BERT

# --- Custom Activation Layers (OptimA and OptimALinear) ---
class OptimA(Layer):
    """Custom Optimal Activation function."""
    def __init__(self, **kwargs):
        super(OptimA, self).__init__(**kwargs)

    def build(self, input_shape):
        self.alpha = self.add_weight(name='alpha', shape=(), initializer='ones', trainable=True)
        self.beta = self.add_weight(name='beta', shape=(), initializer=tf.keras.initializers.Constant(0.5), trainable=True)
        self.gamma = self.add_weight(name='gamma', shape=(), initializer='ones', trainable=True)
        self.delta = self.add_weight(name='delta', shape=(), initializer=tf.keras.initializers.Constant(0.5), trainable=True)
        self.lambda_ = self.add_weight(name='lambda', shape=(), initializer='ones', trainable=True)
        super(OptimA, self).build(input_shape)

    def call(self, x):
        term1 = self.alpha * tf.math.tanh(self.beta * x)
        term2 = self.gamma * tf.math.softplus(self.delta * x) * tf.math.sigmoid(self.lambda_ * x)
        return term1 + term2

    def get_config(self):
        config = super(OptimA, self).get_config()
        return config

class OptimALinear(Layer):
    """Custom Optimal Activation function (Linear Approximation)."""
    def __init__(self, epsilon=1e-5, **kwargs):
        super(OptimALinear, self).__init__(**kwargs)
        self.epsilon = epsilon

    def build(self, input_shape):
        self.alpha = self.add_weight(name='alpha', shape=(), initializer='ones', trainable=True)
        self.beta = self.add_weight(name='beta', shape=(), initializer=tf.keras.initializers.Constant(0.5), trainable=True)
        self.gamma = self.add_weight(name='gamma', shape=(), initializer='ones', trainable=True)
        self.delta = self.add_weight(name='delta', shape=(), initializer=tf.keras.initializers.Constant(0.5), trainable=True)
        self.lambda_ = self.add_weight(name='lambda', shape=(), initializer='ones', trainable=True)
        super(OptimALinear, self).build(input_shape)

    def call(self, x):
        term1 = self.alpha * tf.clip_by_value(self.beta * x, -1, 1)
        term2 = self.gamma * (tf.maximum(0.0, self.delta * x) + self.epsilon) * (0.5 + 0.25 * self.lambda_ * x)
        return term1 + term2

    def get_config(self):
        config = super(OptimALinear, self).get_config()
        config.update({'epsilon': self.epsilon})
        return config

def set_seed(seed=42):
    """Sets random seeds for reproducibility."""
    tf.random.set_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    # tf.config.threading.set_inter_op_parallelism_threads(1) # Can slow down execution
    # tf.config.threading.set_intra_op_parallelism_threads(1) # Can slow down execution

set_seed(RAND_SEED)

# --- Helper to get activation layer ---
def get_activation(act_config):
    if isinstance(act_config, str): return tf.keras.layers.Activation(act_config)
    elif isinstance(act_config, Layer):
        if type(act_config) == OptimA: return OptimA()
        if type(act_config) == OptimALinear: return OptimALinear()
        return act_config # Should be an instance already
    elif isinstance(act_config, type) and issubclass(act_config, Layer): return act_config()
    else: raise ValueError(f"Unsupported activation: {act_config}")

# --- Data Loading and Preprocessing (NLP) ---
def load_and_prepare_nlp_data(dataset_name, tokenizer, max_length, batch_size, seed):
    """Loads and prepares NLP dataset (e.g., IMDB) using Hugging Face datasets."""
    print(f"Loading dataset: {dataset_name}")
    if dataset_name == "imdb":
        raw_ds = hf_datasets.load_dataset("imdb", trust_remote_code=True)
        text_field = "text"
        label_field = "label"
        num_classes = raw_ds["train"].features[label_field].num_classes
    elif dataset_name == "ag_news":
        raw_ds = hf_datasets.load_dataset("ag_news", trust_remote_code=True)
        text_field = "text"
        label_field = "label"
        num_classes = raw_ds["train"].features[label_field].num_classes
    else:
        raise ValueError(f"Unsupported NLP dataset: {dataset_name}")

    def tokenize_function(examples):
        return tokenizer(examples[text_field], truncation=True, padding='max_length', max_length=max_length)

    print("Tokenizing dataset...")
    tokenized_ds = raw_ds.map(tokenize_function, batched=True)
    tokenized_ds = tokenized_ds.remove_columns([text_field])
    tokenized_ds = tokenized_ds.rename_column(label_field, "labels")

    # 'token_type_ids' is included because the tokenizer generates it,
    # and TFBertModel can use it (defaults to zeros if not provided to the model call).
    # set_format expects all columns that will be converted to TensorFlow tensors.
    columns_to_format = ["input_ids", "attention_mask", "token_type_ids", "labels"]
    tokenized_ds.set_format("tensorflow", columns=columns_to_format)

    def as_tf_dataset(split, shuffle=False):
        # Explicitly create a dictionary for from_tensor_slices
        data_for_slicing = {col: tokenized_ds[split][col] for col in columns_to_format}

        ds = tf.data.Dataset.from_tensor_slices(data_for_slicing)

        if shuffle:
            ds = ds.shuffle(buffer_size=len(tokenized_ds[split]), seed=seed, reshuffle_each_iteration=True)

        ds = ds.batch(batch_size)

        # Map function: selects necessary inputs for the model and one-hot encodes labels
        def restructure_and_one_hot(batch_element):
            model_inputs = {
                'input_ids': batch_element['input_ids'],
                'attention_mask': batch_element['attention_mask']
                # 'token_type_ids' can be optionally passed here if the model is built to expect them explicitly.
                # TFBertModel uses them internally if they are part of the output signature from the tokenizer.
            }
            one_hot_labels = tf.one_hot(batch_element['labels'], depth=num_classes)
            return model_inputs, one_hot_labels

        ds = ds.map(restructure_and_one_hot, num_parallel_calls=tf.data.AUTOTUNE)
        return ds.prefetch(tf.data.AUTOTUNE)

    tf_train_dataset = as_tf_dataset("train", shuffle=True)

    if "validation" in tokenized_ds:
        tf_eval_dataset = as_tf_dataset("validation") # Used for validation during training
        tf_test_dataset = as_tf_dataset("test")     # Used for final evaluation
    elif "test" in tokenized_ds:
        print("Warning: No 'validation' split found. Using 'test' split for evaluation during training and final testing.")
        # If no 'validation' split, 'test' is used for both validation and testing.
        # For rigorous research, separate validation and test splits are recommended.
        tf_eval_dataset = as_tf_dataset("test")
        tf_test_dataset = tf_eval_dataset # Use the same dataset for final evaluation
    else:
        raise ValueError("Dataset must have a 'test' or 'validation' split.")

    print(f"Number of classes for {dataset_name}: {num_classes}")
    return tf_train_dataset, tf_eval_dataset, tf_test_dataset, num_classes

# --- Model Building (NLP with BERT) ---
def build_bert_classifier_model(bert_model_name_local, num_classes_local, activation_func_config, max_length_local, strategy=None):
    if isinstance(activation_func_config, str): activation_name = activation_func_config
    elif hasattr(activation_func_config, '__name__'): activation_name = activation_func_config.__name__
    else: activation_name = activation_func_config.__class__.__name__

    context = strategy.scope() if strategy else tf.device('/CPU:0') # Fallback to CPU if no strategy

    with context:
        # 1. Load pre-trained BERT model
        bert_model_layer = TFBertModel.from_pretrained(bert_model_name_local)
        bert_model_layer.trainable = True # Fine-tune BERT layers

        # Input layers
        input_ids = Input(shape=(max_length_local,), dtype=tf.int32, name='input_ids')
        attention_mask = Input(shape=(max_length_local,), dtype=tf.int32, name='attention_mask')

        # 2. Pass inputs through BERT layer
        bert_outputs = bert_model_layer(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = bert_outputs.pooler_output

        # Custom classifier head
        x = Dropout(0.2, name='dropout_bert_out')(pooled_output)
        x = Dense(256, name='dense_pre_activation')(x)
        x = get_activation(activation_func_config)(x) # Apply configured activation
        x = Dropout(0.2, name='dropout_post_activation')(x)
        output_tensor = Dense(num_classes_local, activation='softmax', name='classifier_output')(x)

        model_inputs = {'input_ids': input_ids, 'attention_mask': attention_mask}
        model = Model(inputs=model_inputs, outputs=output_tensor, name=f"BERT_{activation_name}_Classifier")

    return model

# --- Experiment Execution (adapted for NLP) ---
def run_nlp_experiment(nlp_datasets_config, activations_config, nlp_optimizers_config,
                       num_runs, epochs_nlp_local, batch_size_nlp_local, patience_nlp_local,
                       max_len, bert_model_n_local):
    results = {}
    histories = {}

    tokenizer = BertTokenizer.from_pretrained(bert_model_n_local)

    # Attempt to initialize TPU (if available, e.g., on Kaggle/Colab)
    try:
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect() # connect() if on TPU VM
        strategy = tf.distribute.TPUStrategy(tpu)
        print('Running on TPU:', tpu.resolver.master())
        print(f"Number of accelerators: {strategy.num_replicas_in_sync}")
    except ValueError:
        print("TPU not found. Using default strategy (CPU/GPU).")
        # Use MirroredStrategy if GPUs are available, otherwise default (CPU)
        gpus = tf.config.list_physical_devices('GPU')
        if gpus:
            strategy = tf.distribute.MultiWorkerMirroredStrategy()
            print(f"Running on {len(gpus)} GPU(s).")
        else:
            strategy = tf.distribute.get_strategy() # Default strategy (CPU)
            print("Running on CPU.")
        print(f"Number of accelerators: {strategy.num_replicas_in_sync}")


    for run_idx in range(num_runs):
        print(f"\n--- Starting NLP Run {run_idx + 1}/{num_runs} ---")
        current_seed = RAND_SEED + run_idx
        set_seed(current_seed) # Set seed for each run for reproducibility

        for dataset_name, dataset_cfg in nlp_datasets_config.items():
            print(f"\nNLP Dataset: {dataset_name} (Task: {dataset_cfg['task']})")
            if dataset_name not in results:
                results[dataset_name], histories[dataset_name] = {}, {}

            print(f"Loading and tokenizing {dataset_name} for run {run_idx + 1}...")
            # Pass current_seed to load_and_prepare_nlp_data for reproducible shuffle
            train_ds, val_ds, test_ds, num_classes_ds = load_and_prepare_nlp_data(
                dataset_name, tokenizer, max_len, batch_size_nlp_local, current_seed
            )

            loss_f, eval_m, primary_m_name, primary_m_idx = 'categorical_crossentropy', ['accuracy'], 'accuracy', 1

            for opt_name, opt_creator in nlp_optimizers_config.items():
                print(f"  Optimizer: {opt_name} (Run {run_idx + 1})")
                if opt_name not in results[dataset_name]:
                    results[dataset_name][opt_name], histories[dataset_name][opt_name] = {}, {}

                for act_name, act_cfg_val in activations_config.items():
                    print(f"    Activation: {act_name}")
                    if act_name not in results[dataset_name][opt_name]:
                        results[dataset_name][opt_name][act_name] = {'loss': [], primary_m_name: [], 'time': [], 'params': []}
                        histories[dataset_name][opt_name][act_name] = []

                    tf.keras.backend.clear_session(); gc.collect(); set_seed(current_seed)

                    # Build and compile model within strategy.scope()
                    with strategy.scope():
                        model = build_bert_classifier_model(bert_model_n_local, num_classes_ds, act_cfg_val, max_len, strategy)
                        opt_instance = opt_creator()
                        model.compile(optimizer=opt_instance, loss=loss_f, metrics=eval_m)
                    # model.summary() # For debugging

                    # Callbacks
                    mon_metric = 'val_accuracy' # Metric for EarlyStopping and ReduceLROnPlateau
                    cbs = [
                        EarlyStopping(monitor=mon_metric, patience=patience_nlp_local, restore_best_weights=True, verbose=1),
                        ReduceLROnPlateau(monitor=mon_metric, factor=0.2, patience=patience_nlp_local // 2 + 1, min_lr=1e-7, verbose=1)
                    ]

                    start_t = time.time()
                    print(f"Starting training: {dataset_name}/{opt_name}/{act_name}...")
                    hist = model.fit(
                        train_ds,
                        epochs=epochs_nlp_local,
                        validation_data=val_ds, # Use val_ds for validation during training
                        callbacks=cbs,
                        verbose=1
                    )
                    train_time = time.time() - start_t

                    print(f"Evaluating model on test set: {dataset_name}/{opt_name}/{act_name}...")
                    # Evaluate on the separate test_ds
                    eval_res = model.evaluate(test_ds, verbose=0, return_dict=False) # return_dict=False for list output

                    eval_l, eval_pm = eval_res[0], eval_res[primary_m_idx]
                    n_params = model.count_params()

                    res_dict = results[dataset_name][opt_name][act_name]
                    res_dict['loss'].append(eval_l); res_dict[primary_m_name].append(eval_pm)
                    res_dict['time'].append(train_time); res_dict['params'].append(n_params)
                    histories[dataset_name][opt_name][act_name].append(hist.history)
                    print(f"      Test Loss: {eval_l:.4f}, Test {primary_m_name.capitalize()}: {eval_pm:.4f}, Time: {train_time:.2f}s, Params: {n_params}")

                    del model, opt_instance, hist; gc.collect() # Clean up model and history
            del train_ds, val_ds, test_ds; gc.collect() # Clean up dataset data
    return results, histories

# --- Aggregate and Display Results ---
def aggregate_and_display_results(final_results, datasets_cfg_local, all_histories_local, num_runs_executed):
    """Aggregates results from multiple runs and displays them."""
    aggregated = {}
    best_metric_overall = {} # Store best primary metric for each dataset

    # First pass: aggregate all data
    for ds_name, opt_data in final_results.items():
        aggregated[ds_name] = {}
        task = datasets_cfg_local[ds_name]['task'] # 'classification' or 'regression'
        # Determine primary metric based on task
        primary_metric_key = 'accuracy' if task == 'classification' else 'mae' # Adapt if using other regression metrics
        best_metric_overall[ds_name] = {'value': 0 if task == 'classification' else float('inf'),
                                        'optimizer': None, 'activation': None,
                                        'is_higher_better': task == 'classification'}


        for opt_n, act_data in opt_data.items():
            aggregated[ds_name][opt_n] = {}
            for act_n, metrics_vals in act_data.items():
                if not metrics_vals['loss']: # Handle cases where a run might have failed or not produced results
                    agg_data = {'Avg Loss': np.nan, 'Std Loss': np.nan,
                                f'Avg {primary_metric_key.capitalize()}': np.nan,
                                f'Std {primary_metric_key.capitalize()}': np.nan,
                                'Avg Time': np.nan, 'Std Time': np.nan, 'Avg Params': np.nan}
                else:
                    avg_primary_metric = np.mean(metrics_vals[primary_metric_key])
                    agg_data = {
                        'Avg Loss': np.mean(metrics_vals['loss']),
                        'Std Loss': np.std(metrics_vals['loss']),
                        f'Avg {primary_metric_key.capitalize()}': avg_primary_metric,
                        f'Std {primary_metric_key.capitalize()}': np.std(metrics_vals[primary_metric_key]),
                        'Avg Time': np.mean(metrics_vals['time']),
                        'Std Time': np.std(metrics_vals['time']),
                        'Avg Params': int(np.mean(metrics_vals['params'])) if metrics_vals['params'] else np.nan
                    }

                    # Track best metric for this dataset
                    is_better = (best_metric_overall[ds_name]['is_higher_better'] and avg_primary_metric > best_metric_overall[ds_name]['value']) or \
                                (not best_metric_overall[ds_name]['is_higher_better'] and avg_primary_metric < best_metric_overall[ds_name]['value'])
                    if is_better:
                        best_metric_overall[ds_name]['value'] = avg_primary_metric
                        best_metric_overall[ds_name]['optimizer'] = opt_n
                        best_metric_overall[ds_name]['activation'] = act_n

                aggregated[ds_name][opt_n][act_n] = agg_data

    # Create DataFrame for results
    all_rows = []
    for ds, o_data in aggregated.items():
        task = datasets_cfg_local[ds]['task']
        primary_metric_key_cap = ('Accuracy' if task == 'classification' else 'Mae') # Adapt if metric is different

        for o, a_data in o_data.items():
            for a, m_vals in a_data.items():
                row = {'Dataset': ds, 'Optimizer': o, 'Activation': a}
                row.update(m_vals)

                # Add marker for the best combination for this dataset
                is_best_current_combo = (best_metric_overall.get(ds, {}).get('optimizer') == o and
                                         best_metric_overall.get(ds, {}).get('activation') == a)
                row[f'Best {primary_metric_key_cap}'] = '★' if is_best_current_combo else ''
                all_rows.append(row)

    df_res = pd.DataFrame(all_rows)
    if not df_res.empty:
        df_res = df_res.set_index(['Dataset', 'Optimizer', 'Activation'])
        # Dynamically determine column order based on available metrics
        cols_order = []
        for m_cap in ['Accuracy', 'Mae']: # Add other primary metrics if used
            if f'Best {m_cap}' in df_res.columns: cols_order.append(f'Best {m_cap}')
        for m_cap in ['Accuracy', 'Mae']:
            if f'Avg {m_cap}' in df_res.columns: cols_order.append(f'Avg {m_cap}')
        for m_cap in ['Accuracy', 'Mae']:
            if f'Std {m_cap}' in df_res.columns: cols_order.append(f'Std {m_cap}')

        cols_order += ['Avg Loss', 'Std Loss', 'Avg Time', 'Std Time', 'Avg Params']
        cols_order = [col for col in cols_order if col in df_res.columns] # Ensure only existing columns are used
        df_res = df_res.reindex(columns=cols_order).sort_index()

    print("\n--- Aggregated NLP Benchmark Results ---")
    if best_metric_overall:
        print("\nBest Performing Combinations per Dataset:")
        for ds, info in best_metric_overall.items():
            metric_name = 'Accuracy' if info['is_higher_better'] else 'MAE' # Or other relevant metric name
            print(f"  {ds}: {info['optimizer']} with {info['activation']} ({metric_name}: {info['value']:.4f})")

    pd.options.display.float_format = '{:.4f}'.format
    print(df_res.dropna(axis=1, how='all') if not df_res.empty else "No NLP results to display.")

# --- Main Execution (NLP) ---
if __name__ == "__main__":
    # Ensure internet access for downloading Hugging Face models and datasets
    os.environ["HF_HUB_DISABLE_PROGRESS_BARS"] = "1" # Disable HF progress bars if desired

    # Define NLP datasets for the benchmark
    nlp_datasets_run_cfg = {
        "imdb": {"task": "classification"}, # Using 'classification' as the task type
        # "ag_news": {"task": "classification"}, # More datasets can be added if time/resources permit
    }
    if not nlp_datasets_run_cfg:
        print("No NLP datasets configured. Exiting.")
        exit()

    # Activation functions
    activations_run_cfg = {
        'OptimA': OptimA,
        'OptimALinear': OptimALinear,
        'GeLU': 'gelu', # GeLU is a standard activation in BERT
    }

    # Optimizers for BERT fine-tuning
    nlp_optimizers_run_cfg = {
        # AdamW is standard for BERT. Epsilon is often 1e-8 for Transformers.
        'AdamW_BERT': lambda: AdamW(learning_rate=LEARNING_RATE_BERT, epsilon=1e-8, beta_1=0.95, beta_2=0.999, amsgrad=True),
        # Other AdamW variations or optimizers can be added for comparison if desired
    }

    print(f"--- NLP BENCHMARK ---")
    print(f"BERT Model: {BERT_MODEL_NAME}, Max Sequence Length: {MAX_LENGTH}")
    print(f"Effective Batch Size: {BATCH_SIZE_NLP}, Epochs: {EPOCHS_NLP}, Number of Runs: {NUM_RUNS}")


    # Run NLP experiments
    final_nlp_data, all_nlp_hists = run_nlp_experiment(
        nlp_datasets_run_cfg, activations_run_cfg, nlp_optimizers_run_cfg,
        num_runs=NUM_RUNS, epochs_nlp_local=EPOCHS_NLP, batch_size_nlp_local=BATCH_SIZE_NLP,
        patience_nlp_local=PATIENCE_NLP, max_len=MAX_LENGTH, bert_model_n_local=BERT_MODEL_NAME
    )

    # Display aggregated NLP results
    aggregate_and_display_results(final_nlp_data, nlp_datasets_run_cfg, all_nlp_hists, NUM_RUNS)

2025-05-10 16:10:12.658963: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-05-10 16:10:12.658993: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-05-10 16:10:12.659955: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-10 16:10:12.665139: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-05-10 16:10:13.189866: W tensorflow/compiler/tf2

--- NLP BENCHMARK ---
BERT Model: bert-base-uncased, Max Sequence Length: 128
Effective Batch Size: 16, Epochs: 3, Number of Runs: 1


/home/sad/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


TPU not found. Using default strategy (CPU/GPU).


2025-05-10 16:10:15.881493: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-05-10 16:10:15.889849: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-05-10 16:10:15.923143: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

...

1562/1563 [============================>.] - ETA: 6s - loss: 0.0978 - accuracy: 0.9640 

1563/1563 [==============================] - ETA: 0s - loss: 0.0978 - accuracy: 0.9640


Epoch 3: ReduceLROnPlateau reducing learning rate to 3.999999898951501e-06.


1563/1563 [==============================] - 13303s 9s/step - loss: 0.0978 - accuracy: 0.9640 - val_loss: 0.3645 - val_accuracy: 0.8845 - lr: 2.0000e-05


Evaluating model on test set: imdb/AdamW_BERT/OptimA...
